## The Challenge

We are an asset manager who has outsourced our fund accounting. We have multiple clients, each with multiple portfolios containing different investment strategies and priorities.

We are feeling a lot of fee pressure and to maintain profitability we need to keep our costs low.

Outsourcing our fund accounting was a big part of this plan, however there have been a large number of significant discrepancies between our records and those of the fund accountant. Thus we have had to set up an internal 'shadow' team to reconcile the differences between our records and those of the fund accountant. The cost of running this team almost completely negates the cost saving of outsourcing our fund accounting. In addition, there have been many times were we have just had to accept the records of the fund accountant as being correct.

This has lead our portfolio managers to become incredibly frustrated as they can't accurately determine their position and are being forced into trading on the basis of unreliable information.

We desperately need a solution that allows us to improve our reconciliation process so that we can easily identify & reconcile the discrepancies between our records and those of the fund accountant.

## The Solution

LUSID has been designed to help address challenges such as this.

Using LUSID we can automate the reconciliation processs to help our investment managers get a better understanding of their position and remove the need to have a separate shadow accounting team.

We can do this by:

1. Creating one environment for our internal records and one for those of the fund accountant
2. Creating the current holdings for our clients in both environments
3. Completing a day of trading and updating our internal records with the day's trades
4. Receiving a daily holdings report from our fund accountant and updating the fund accountant record
5. Reconciling our internal records with those of the fund accountant
6. Building a system to automatically identify and flag any discrepancies

At the end of this process our LUSID environment will look like the below:

![Scopes](img/transparency-matchedexceptions.gif)

In [1]:
# Import LUSID
import lusid.models as models
import lusid_sample_data as import_data

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print ('LUSID Environment Initialised')
print ('LUSID API Version :', client.metadata.get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID API Version : 0.5.2891.0


# 1) Create our client portfolios in an internal and fund accountant environment

### a) Create our Scopes

In LUSID we can create separate environments for our records and those of the fund accountant using scopes. A scope is a container for LUSID objects and can be thought of as a separate identity namespace. This allows us to have a portfolio with the same name in an 'internal-records' scope as well as in a 'fund-accountant-records' scope without having to worry about collisions.

We will thus use two scopes, one for our internal records and one for the fund accountant's records.

*Run the cell below to initialise our two scopes. Note that each scope will have a unique 4 character code appended to it to make the name more unique.*

In [2]:
# Fetch our scopes
scope_id = import_data.create_scope_id()
internal_scope_code = 'internal-records-{}'.format(scope_id)
scope_id = import_data.create_scope_id()
fund_accountant_scope_code = 'fund-accountant-records-{}'.format(scope_id)

scopes = [internal_scope_code, fund_accountant_scope_code]

prettyprint.heading('Internal Scope Code', internal_scope_code)
prettyprint.heading('Fund Accountant Scope Code', fund_accountant_scope_code)

Internal Scope Code: internal-records-378c-6794-3518-5a
Fund Accountant Scope Code: fund-accountant-records-378c-6794-3521-3a


Our LUSID environment now looks like the below. We will update this with each step.

![Scopes](img/transparency-scopes.gif)

### b) Create our portfolios 

Now that we have our scopes we can create our portfolios.

The first thing that we need to do here is give each portfolio a unique name. In LUSID these are refered to as 'codes'. Almost every object in LUSID can be refered to by the combination of the scope that it is contained in and its code. The code must be unique to the scope.

We have three clients each with a varying number of portfolios. Each portfolio has a different objective to meet the client's needs. For example a client may have a balanced portfolio which allows them to generate a sustained and predictable return and a fixed income portfolio to meet their distribution requirements.

Let us call these clients:

- Client A
- Client B
- Client C

In LUSID we can use portfolio groups to group the different portfolios of each of our clients together. This allows us to easily keep track of and work with the holdings of each of our clients.

*Run the cell below to import the names for our portfolio groups and their corresponding portfolios.*

In [3]:
client_portfolios = import_data.fetch_portfolio_names('portfolios.csv')
prettyprint.portfolios(client_portfolios)

Portfolio group: client-A-portfolios
   Portfolio: client-A-strategy-balanced
   Portfolio: client-A-strategy-tech
   Portfolio: client-A-strategy-growth


Portfolio group: client-B-portfolios
   Portfolio: client-B-strategy-balanced
   Portfolio: client-B-strategy-energy
   Portfolio: client-B-strategy-fixedincome
   Portfolio: client-B-strategy-international
   Portfolio: client-B-strategy-usgovt


Portfolio group: client-C-portfolios
   Portfolio: client-C-strategy-balanced
   Portfolio: client-C-strategy-fixedincome




Now that we have the codes for our portfolios, let us create them in LUSID across both our internal and fund accountant scopes.

We can only create one portfolio at a time so what we will do is create each of the portfolios in a group and then once they have all been created create the group binding them all together.

Note that we have had these clients for a while and so the portfolios have actually been live for 2 years, they just haven't been added to LUSID yet. when we create them we will specify the date they went live using the 'created' parameter.

This allows us to backdate trades. If we don't specify the correct created date we won't be able to add any trades before today's date.

When we create a portfolio we will specify the following, note that not all of these fields are compulsory.

- Display Name
- Code
- Base Currency
- Description
- Creation Date

*Run the cell below to create our portfolios and their portfolio groups*

In [4]:
# The date our portfolios were first created
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=895)).isoformat()

# Iterate over our two scopes, this will create our portfolios and portfolio groups in both scopes
for scope in scopes:
    
    # Iterate over our portfolio groups selecting the name of the group and the list of portfolios
    for portfolio_group_code, portfolio_group in client_portfolios.items():

        # Loop over our list of portfolios selecting the portfolio code
        for portfolio_code in portfolio_group:

            # Create the request to add our portfolio
            portfolio_request = models.CreateTransactionPortfolioRequest(
                display_name=portfolio_code,
                code=portfolio_code,
                base_currency='GBP',
                description=portfolio_code,
                created=portfolio_creation_date)

            # Create our portfolio
            response = client.transaction_portfolios.create_portfolio(
                scope=scope,
                create_request=portfolio_request)
            
            prettyprint.portfolio_response(response)


        '''
        Now that we have created all of the portfolios for this client we can create our portfolio group and add the
        portfolios to it. To add a portfolio to a group we refer to it by its LUSID ResourceId. This is simply the
        scope and code of the portfolio
        '''

        # Prepare a list of ResourceIDs for our portfolios
        portfolio_resource_ids = [
            models.ResourceId(
                scope=scope,
                code=portfolio_code) for portfolio_code in portfolio_group]

        # Build our request
        portfolio_group_request = models.CreatePortfolioGroupRequest(
            code=portfolio_group_code,
            display_name=portfolio_group_code,
            values=portfolio_resource_ids,
            description=portfolio_group_code)
        
        # Create our portfolio group
        response = client.portfolio_groups.create_portfolio_group(
            scope=scope,
            request=portfolio_group_request)
        
        prettyprint.portfolio_group_response(response, 'created')
        
print ('All portfolios have been created')

Portfolio Created
Scope: internal-records-378c-6794-3518-5a
Code: client-A-strategy-balanced
Portfolio Effective From: 2017-02-04 15:27:01.553494+00:00
Portfolio Created On: 2019-07-19 15:27:02.252563+00:00

Portfolio Created
Scope: internal-records-378c-6794-3518-5a
Code: client-A-strategy-tech
Portfolio Effective From: 2017-02-04 15:27:01.553494+00:00
Portfolio Created On: 2019-07-19 15:27:02.491531+00:00

Portfolio Created
Scope: internal-records-378c-6794-3518-5a
Code: client-A-strategy-growth
Portfolio Effective From: 2017-02-04 15:27:01.553494+00:00
Portfolio Created On: 2019-07-19 15:27:02.701780+00:00

Portfolio Group Created
Name: client-A-portfolios
Scope: internal-records-378c-6794-3518-5a
Code: client-A-portfolios
Portfolios Inside Group: 
client-A-strategy-balanced
client-A-strategy-tech
client-A-strategy-growth


Portfolio Created
Scope: internal-records-378c-6794-3518-5a
Code: client-B-strategy-balanced
Portfolio Effective From: 2017-02-04 15:27:01.553494+00:00
Portfolio

Our LUSID environment is now populated with our client's portfolios in both of our scopes. These portfolios are currently empty.

![Scopes](img/transparency-portfolios.gif)

# 2) Create our current holdings for our clients 

Now that we have our portfolios and groups set up for our clients we can add in their current holdings. We will populate the same holdings for both the internal scope and the fund accountant scope. This will allow us to start with both records completely in sync with each other.

### a) Create our instrument universe

Before we can set our holdings we first need to upsert our instrument into LUSID.

In LUSID we use an upsert method. As opposed to Insert, Upsert will insert a new record if it does not exist and update the existing record if it does exist. This allows us to add instruments more simply than checking if an instrument exists before we insert it.

*Run the cell below to import our instrument universe. This will return a name for each of our instruments and the internal ID we have assigned to it*

In [5]:
instrument_universe = import_data.fetch_instrument_universe('instruments.csv')
prettyprint.instruments(instrument_universe, 'ClientInternal')

Amazon_Nasdaq_AMZN: ClientInternal - imd_34634534
Apple_Nasdaq_AAPL: ClientInternal - imd_35345345
BP_LondonStockEx_BP: ClientInternal - imd_43535553
BurfordCapital_LondonStockEx_BUR: ClientInternal - imd_43534356
EKFDiagnostics_LondonStockEx_EKF: ClientInternal - imd_34535355
Glencore_LondonStockEx_GLEN: ClientInternal - imd_34534555
JustEat_LondonStockEx_JE: ClientInternal - imd_35436366
Kingfisher_LondonStockEx_KGF: ClientInternal - imd_34535552
MicroFocus_LondonStockEx_MCRO: ClientInternal - imd_34567338
RELXGroup_LondonStockEx_REL: ClientInternal - imd_43532542
Sage_LondonStockEx_SGE: ClientInternal - imd_23536673
TESCO_LondonStockEx_TSCO: ClientInternal - imd_34634673
UKGiltTreasury_2.0_2025: ClientInternal - imd_34534536
UKGiltTreasury_3.5_2045: ClientInternal - imd_54234532
UKGiltTreasury_3.75_2021: ClientInternal - imd_34643653
UKGiltTreasury_4.5_2034: ClientInternal - imd_34534534
USTreasury_2.00_2021: ClientInternal - imd_34535347
USTreasury_6.875_2025: ClientInternal - imd_

Now that we have our instruments, we can upsert our instruments individually or in a batch. Either way the request should be in the form of key value pairs. We will use the instrument's name as the key and our request to upsert the instrument as the value.

The request requires a name for the instrument which is a string. In this case we will use the same name as our key.

It also requires at least one unique identifier for the instrument. This again takes the form of a key value pair. In this case we will use 'ClientInternal' as the key. The value is a string which is the identifier. The allowed unique identifiers currently supported are:

* ClientInternal
* Figi
* RIC
* QuotePermId

Note that many other non-unqiue identifers such as ISIN, SEDOL etc. are also supported. However as these are not necessarily unique and may be re-used over time they are unsuitable as unique identifers for our instruments. Instead we can add them as properties.

Note that unlike most LUSID objects such as our portfolios, instrumets are global to your account and are not contained within a scope.

*Run the cell below to upsert our instruments into LUSID*

In [6]:
# Initialise our batch upsert request
batch_upsert_request = {}
# Using our instrument universe create our batch request
for instrument_name, instrument in instrument_universe.items():
    batch_upsert_request[instrument_name] = models.InstrumentDefinition(
        name=instrument_name,
        identifiers={
            'ClientInternal': models.InstrumentIdValue(
                value=instrument['identifiers']['ClientInternal'])
        }
    )
    
# Upsert our batch
instrument_response = client.instruments.upsert_instruments(
    requests=batch_upsert_request)

prettyprint.instrument_response(instrument_response)

Instrument Successfully Upserted: BurfordCapital_LondonStockEx_BUR
ClientInternal ID: imd_43534356
LUSID Instrument ID: LUID_KAOYB8MW


Instrument Successfully Upserted: Whitebread_LondonStockEx_WTB
ClientInternal ID: imd_35349900
LUSID Instrument ID: LUID_EXSJHHWE


Instrument Successfully Upserted: Kingfisher_LondonStockEx_KGF
ClientInternal ID: imd_34535552
LUSID Instrument ID: LUID_OARPBV3G


Instrument Successfully Upserted: UKGiltTreasury_4.5_2034
ClientInternal ID: imd_34534534
LUSID Instrument ID: LUID_6P89F05F


Instrument Successfully Upserted: Amazon_Nasdaq_AMZN
ClientInternal ID: imd_34634534
LUSID Instrument ID: LUID_RYABIZUF


Instrument Successfully Upserted: UKGiltTreasury_3.5_2045
ClientInternal ID: imd_54234532
LUSID Instrument ID: LUID_OLABWDN8


Instrument Successfully Upserted: USTreasury_2.00_2021
ClientInternal ID: imd_34535347
LUSID Instrument ID: LUID_RZ3USX5U


Instrument Successfully Upserted: TESCO_LondonStockEx_TSCO
ClientInternal ID: imd_34634673
LUSID Ins

Our LUSID environment now has an instrument master containing all of our instruments that we will be trading.

![Scopes](img/transparency-instrumentmaster.gif)

During the upsertion process you may have noticed that that every instrument that is created in LUSID given a unique LUSID Instrument Id or LUID for short. This ID is used for many methods and is how LUSID uniquely identifies an instrument.

Note that there is also a match feature that allows you to search for instruments using the identifiers that we defined for each instrument, however it is preferred to use the LUID as this is guaranteed to be unique and there is absolutely no chance of a collision.

We therefore want to add our newly created LUIDs to our instrument list so that we can easily reference our instruments in future.

Note that in addition to our instruments, there is also cash in our holdings. Instruments that represent cash are already set-up in LUSID by default. In LUSID the identifier for cash is the ISO4217 currency code. For example British Pounds have an identifier of GBP. We will add these to our local instrument universe so that we can reference them.

*Run the cell below to add our LUIDs and cash to our local instrument universe*

In [7]:
# Loop over our recently upserted instruments
for instrument_name, instrument in instrument_response.values.items():
    # Add our LUID as a new identifier so that we can use it in our calls later
    instrument_universe[instrument_name]['identifiers']['LUID'] = instrument.lusid_instrument_id
    
# We will also add some cash instruments to our universe
instrument_universe['GBP_Cash'] = {
    'identifiers': {'LUID': 'GBP'},
    'currency': 'GBP'}

instrument_universe['USD_Cash'] = {
    'identifiers': {'LUID': 'USD'},
    'currency': 'USD'}
            
prettyprint.instruments(instrument_universe, 'LUID')

Amazon_Nasdaq_AMZN: LUID - LUID_RYABIZUF
Apple_Nasdaq_AAPL: LUID - LUID_BX8AC7YF
BP_LondonStockEx_BP: LUID - LUID_FZYQSI3S
BurfordCapital_LondonStockEx_BUR: LUID - LUID_KAOYB8MW
EKFDiagnostics_LondonStockEx_EKF: LUID - LUID_LSGPYIQC
Glencore_LondonStockEx_GLEN: LUID - LUID_KLALKNU6
JustEat_LondonStockEx_JE: LUID - LUID_JBRAHMOH
Kingfisher_LondonStockEx_KGF: LUID - LUID_OARPBV3G
MicroFocus_LondonStockEx_MCRO: LUID - LUID_W15NRGU1
RELXGroup_LondonStockEx_REL: LUID - LUID_ODRIPEQV
Sage_LondonStockEx_SGE: LUID - LUID_6MEXFDK7
TESCO_LondonStockEx_TSCO: LUID - LUID_ANQFWLK4
UKGiltTreasury_2.0_2025: LUID - LUID_EI0UW9L6
UKGiltTreasury_3.5_2045: LUID - LUID_OLABWDN8
UKGiltTreasury_3.75_2021: LUID - LUID_VHZLQK26
UKGiltTreasury_4.5_2034: LUID - LUID_6P89F05F
USTreasury_2.00_2021: LUID - LUID_RZ3USX5U
USTreasury_6.875_2025: LUID - LUID_O0J4DP42
WPP_LondonStockEx_WPP: LUID - LUID_86YWK4X9
Whitebread_LondonStockEx_WTB: LUID - LUID_EXSJHHWE
GBP_Cash: LUID - GBP
USD_Cash: LUID - USD


### b) Set our take on balances

Let us load our take on balances. Note that in practice this data will likely be imported from a CSV or report generated from an internal system or by a third party that summarises our client's holdings.

Each holding as the instrument name, the quantity currently held and the weighted average price of the holding. Note that LUSID can support other more complex accounting methods such as First in First Out (FIFO), Last in First Out (LIFO) etc.

*Run the cell below to import our initial holdings information*

In [8]:
client_holdings = import_data.fetch_client_take_on_balances('holdings.csv')
prettyprint.holdings(client_holdings)

Portfolio: client-A-strategy-balanced

Instrument Name: WPP_LondonStockEx_WPP
Quantity: 2956000
CostPrice: 8.71

Instrument Name: UKGiltTreasury_2.0_2025
Quantity: 375856
CostPrice: 8.71

Instrument Name: JustEat_LondonStockEx_JE
Quantity: 4026354
CostPrice: 5.4639999999999995

Instrument Name: UKGiltTreasury_3.75_2021
Quantity: 486913
CostPrice: 108.126

Instrument Name: GBP_Cash
Quantity: 3000000
CostPrice: 1.0




Portfolio: client-A-strategy-tech

Instrument Name: MicroFocus_LondonStockEx_MCRO
Quantity: 687994
CostPrice: 14.535

Instrument Name: Sage_LondonStockEx_SGE
Quantity: 2599653
CostPrice: 5.77

Instrument Name: GBP_Cash
Quantity: 784000
CostPrice: 1.0




Portfolio: client-A-strategy-growth

Instrument Name: BurfordCapital_LondonStockEx_BUR
Quantity: 853486
CostPrice: 14.06

Instrument Name: EKFDiagnostics_LondonStockEx_EKF
Quantity: 925925
CostPrice: 0.27

Instrument Name: GBP_Cash
Quantity: 150000
CostPrice: 1.0




Portfolio: client-B-strategy-balanced

Instrument Name: 

To set our holdings in LUSID we use a list of adjustment requests.

Using these requests LUSID will determine what transactions are required to update our portfolios so that they have these holdings.

Each request object has two fields these are:

* instrument_uid: This is the LUID of the instrument we want to adjust the holdings for

* tax_lots: This is a list of tax lots. A tax lot being instruments purchased at different times and thus needing different treatment for tax purposes. In this case we will use a single tax lot

    * units: This is the quantity of the instrument that we are holding in this tax lot

    * cost: This is the total cost of the instrument and the currency it is in for this tax lot
        * amount: The total cost/value of the instrument, here we multiply the instrument's price by its quantity to get the total amount
        * currency: The currency that the instrument is in as an ISO4217 currency code

    * portfolio_cost: This is the total cost of the instrument in the portfolio's currency for this tax lot

    * price: This is the price of the instrument in the instrument's currency for this tax lot

We will make our holdings effecitve as of two days ago. This gives us room to have a day of trading before we reconcile our internal records with those of the fund accountant.

Just like when we created our portfolios we can only adjust the holdings one portfolio at a time. We will loop over our portfolios to create our adjustment requests before making our adjustments across our internal-records scope and our fund-accountant-records scope.

*Run the cell below to create our requests and adjust our holdings*

In [9]:
holdings_effective_date = (datetime.now(pytz.UTC) - timedelta(days=2)).isoformat()

# Iterate over our portfolios
for portfolio_name, portfolio in client_holdings.items():
    
    # Initialise our list to hold the holding adjustments
    holding_adjustments = []

    # Iterate over the holdings in each portfolio
    for instrument_name, holding in portfolio.items():
        # Get our Lusid Instrument ID
        if 'Cash' in instrument_name:
            identifier_key = 'Instrument/default/Currency'
        else:
            identifier_key = 'Instrument/default/LusidInstrumentId'
            
        Luid = instrument_universe[instrument_name]['identifiers']['LUID']
        # Create our adjust holdings request using our instrument universe to get the LUID identifier for the instrument
        holding_adjustments.append(
            models.AdjustHoldingRequest(
                instrument_identifiers={
                    identifier_key: Luid},
                tax_lots=[
                    models.TargetTaxLotRequest(
                        units=holding['quantity'],
                        cost=models.CurrencyAndAmount(
                           amount=holding['quantity'] *
                                  holding['price'],
                           currency=instrument_universe[instrument_name]['currency']),
                        portfolio_cost=holding['quantity'] *
                                      holding['price'],
                        price=holding['price'])
                ])
        )

    # Iterate over our two scopes
    for scope in scopes:

        set_holdings_response = client.transaction_portfolios.set_holdings(
            scope=scope,
            code=portfolio_name,
            effective_at=holdings_effective_date,
            holding_adjustments=holding_adjustments)

        prettyprint.set_holdings_response(set_holdings_response, scope, portfolio_name)

print ('All portfolios have been set')

Holdings Successfully Set for Portfolio
Scope: internal-records-378c-6794-3518-5a
Code: client-A-strategy-balanced
Holdings Effective From: 2019-07-17 15:27:07.733726+00:00
Holdings Created On: 2019-07-19 15:27:08.702589+00:00

Holdings Successfully Set for Portfolio
Scope: fund-accountant-records-378c-6794-3521-3a
Code: client-A-strategy-balanced
Holdings Effective From: 2019-07-17 15:27:07.733726+00:00
Holdings Created On: 2019-07-19 15:27:09.120542+00:00

Holdings Successfully Set for Portfolio
Scope: internal-records-378c-6794-3518-5a
Code: client-A-strategy-tech
Holdings Effective From: 2019-07-17 15:27:07.733726+00:00
Holdings Created On: 2019-07-19 15:27:09.599799+00:00

Holdings Successfully Set for Portfolio
Scope: fund-accountant-records-378c-6794-3521-3a
Code: client-A-strategy-tech
Holdings Effective From: 2019-07-17 15:27:07.733726+00:00
Holdings Created On: 2019-07-19 15:27:10.009438+00:00

Holdings Successfully Set for Portfolio
Scope: internal-records-378c-6794-3518-5a


Our portfolios are no longer empty and now contain our initial holdings. Both the internal-records scope and the fund-accountant-records scope are in sync.

![Scopes](img/transparency-holdings.gif)

# 3) Complete a day of trading and update our internal records

Now that we have our portfolios populated with their holdings we are going to simulate a day of trading. We are going to add all of yesterdays transactions to our internal-records portfolios.

We had some exciting new strategies to implement across our portfolios that involved

1. Selling stock in WPP
2. Buying more stock in MicroFocus
3. Selling stock in Kingfisher
4. Buying more of the UK Treasury Gilt maturing in 2034 with coupon rate of 4.5%
5. Buying some more UK Treasury Gilts and selling US Treasury Bonds
6. Selling stock in Whitebread
7. Selling some stock in TESCO

In reality these trades would be added to our internal scope as soon as they are executed. They are also sent to our fund manager's systems in real time via API through our execution management system. Note that the fund accountant scope is not updated in real time. Instead we receive a daily report from our fund accountant. We then update the fund accountant scope using this report.

Let us import our trade below. Keeping in mind that these would be generated from our order management or execution management system in reality. Each trade has a unique identifier prefixed with tid_. It also has a transaction date and settlement date.

*Run the cell below to set time that trading opened yesterday and to import our trades*

In [10]:
client_transactions, yesterday_trade_open = import_data.client_transactions('transactions.csv', instrument_universe)
prettyprint.transactions(client_transactions)

Portfolio: client-A-strategy-balanced

Transaction: tid_35b77104-7c72-4515-af2b-d22bb45e6edd
Instrument Name: WPP_LondonStockEx_WPP
Instrument LUID: LUID_86YWK4X9
Type: Sell
Units: 265600
Price: 8.91
Currency: GBP
Transaction Date: 2019-07-18T09:00:00+00:00
Settlement Date: 2019-07-19T08:00:00+00:00



Portfolio: client-A-strategy-tech

Transaction: tid_2ac903c9-3615-4716-80d7-1608d40617dc
Instrument Name: MicroFocus_LondonStockEx_MCRO
Instrument LUID: LUID_W15NRGU1
Type: Buy
Units: 15074
Price: 13.2867
Currency: GBP
Transaction Date: 2019-07-18T13:00:00+00:00
Settlement Date: 2019-07-23T08:00:00+00:00



Portfolio: client-B-strategy-balanced

Transaction: tid_4f192c6a-e74b-464e-95c2-62a18b092bd8
Instrument Name: Kingfisher_LondonStockEx_KGF
Instrument LUID: LUID_OARPBV3G
Type: Sell
Units: 325000
Price: 2.345
Currency: GBP
Transaction Date: 2019-07-18T11:00:00+00:00
Settlement Date: 2019-07-21T08:00:00+00:00

Transaction: tid_6fc1a588-f19b-4222-93d5-c35dcf4c7d46
Instrument Name: UKGilt

Now that we have imported our trades we can create a batch transaction request for each portfolio. This comes in the form of a list of transaction requests.

Each transaction request object has the following properties:

* transaction_id: A unique transaction identifier

* type: The transaction type, by default the following are already available 
    * 'Buy'
    * 'Sell'
    * 'StockIn'
    * 'StockOut'
    * 'CoverShort'
    * 'SellShort'
    * 'FxBuy'
    * 'FxSell'
    * 'FwdFxBuy'
    * 'FwdFxSell'
    * 'FundsIn'
    * 'FundsOut'
    * 'OLFC' (Open Long Futures Contract)
    * 'CLFC' (Close Long Futures Contract)
    * 'OSFC' (Open Short Futures contract)
    * 'CSFC' (Close Short Futures Contract)

* instrument_uid: The Lusid Instrument Id also known as LUID

* transaction_date: The date of the transaction as an ISO8601 datetime, that is "YYYY-MM-DDTHH:MM:SSZ"

* settlement_date: The date of the settlement as an ISO8601 datetime, that is "YYYY-MM-DDTHH:MM:SSZ"

* units: The quantity of units of the instrument involved in the transaction

* transactionPrice: A TransactionPrice object with a price and type
    * price: The price of the transaction
    * type: The type of the price, available options are 'Price', 'Yield' or 'Spread'

* totalConsideration: The total value of the transaction in settlement currency as a CurrencyAndAmount object
    * amount: The amount of the currency
    * currency: The currency as an ISO 4217 currency code

* exchangeRate: Rate between transaction and settlement currency

* transactionCurrency: The currency of the transaction as an ISO 4217 currency code

* source: Where this transaction came from, the options are 'System' and 'Client'

We will iterate over each potfolio to create our requests and then upsert them into LUSID.

*Run the cell below to create our transaction requests and add them to LUSID*

In [11]:
# Iterate over our portfolios
for portfolio_name, portfolio in client_transactions.items():

    # Initialise a list to hold our transactions for each portfolio
    batch_transaction_requests = []

    # Iterate over the transactions for each portfolio
    for transaction_id, transaction in portfolio.items():

        batch_transaction_requests.append(
            models.TransactionRequest(
                transaction_id=transaction_id,
                type=transaction['type'],
                instrument_identifiers={
                    'Instrument/default/LusidInstrumentId': transaction['instrument_uid']},
                transaction_date=transaction['transaction_date'],
                settlement_date=transaction['settlement_date'],
                units=transaction['units'],
                transaction_price=models.TransactionPrice(
                    price=transaction['transaction_price'],
                    type='Price'),
                total_consideration=models.CurrencyAndAmount(
                    amount=transaction['units'] * transaction['transaction_price'],
                    currency=transaction['transaction_currency']),
                source='Client',
                transaction_currency=transaction['transaction_currency'])
        )

    transaction_response = client.transaction_portfolios.upsert_transactions(
        scope=internal_scope_code,
        code=portfolio_name,
        transactions=batch_transaction_requests)

    prettyprint.transactions_response(
        transaction_response, 
        internal_scope_code, 
        portfolio_name)

Transactions Successfully Upserted into Portfolio
Scope: internal-records-378c-6794-3518-5a
Code: client-A-strategy-balanced
Transactions Effective From: 2019-07-18 09:00:00+00:00
Transactions Created On: 2019-07-19 15:27:18.163649+00:00

Transactions Successfully Upserted into Portfolio
Scope: internal-records-378c-6794-3518-5a
Code: client-A-strategy-tech
Transactions Effective From: 2019-07-18 13:00:00+00:00
Transactions Created On: 2019-07-19 15:27:18.451004+00:00

Transactions Successfully Upserted into Portfolio
Scope: internal-records-378c-6794-3518-5a
Code: client-B-strategy-balanced
Transactions Effective From: 2019-07-18 16:12:00+00:00
Transactions Created On: 2019-07-19 15:27:18.920768+00:00

Transactions Successfully Upserted into Portfolio
Scope: internal-records-378c-6794-3518-5a
Code: client-B-strategy-fixedincome
Transactions Effective From: 2019-07-18 12:00:00+00:00
Transactions Created On: 2019-07-19 15:27:19.349188+00:00

Transactions Successfully Upserted into Portf

We can now see that in LUSID the portfolios that have had transactions added to them are no longer in sync with our fund accountant records. These portfolios are in pink in the image below.

![Scopes](img/transparency-transactions.gif)

# 4) Receive our daily holdings report and update the fund accountant record

It is now early in the morning the day after our recent trades. Our fund accountant has just sent us the daily report with our current position according to their records.

We will update our fund accountant scope with their records.

*Run the cell below to import the fund accountants records*

In [12]:
fund_accountant_daily_holdings_report = import_data.fetch_fund_accountant_daily_holdings_report('fundaccountantreport.csv')
prettyprint.holdings(fund_accountant_daily_holdings_report)

Portfolio: client-A-strategy-balanced

Instrument Name: WPP_LondonStockEx_WPP
Quantity: 2690400.0
CostPrice: 8.71

Instrument Name: UKGiltTreasury_2.0_2025
Quantity: 375856.0
CostPrice: 8.71

Instrument Name: JustEat_LondonStockEx_JE
Quantity: 4026354.0
CostPrice: 5.4639999999999995

Instrument Name: UKGiltTreasury_3.75_2021
Quantity: 486913.0
CostPrice: 108.126

Instrument Name: GBP_Cash
Quantity: 5366496.0
CostPrice: 1.0




Portfolio: client-A-strategy-tech

Instrument Name: MicroFocus_LondonStockEx_MCRO
Quantity: 703068.0
CostPrice: 14.5082

Instrument Name: Sage_LondonStockEx_SGE
Quantity: 2599653.0
CostPrice: 5.77

Instrument Name: GBP_Cash
Quantity: 583716.2842
CostPrice: 1.0




Portfolio: client-A-strategy-growth

Instrument Name: BurfordCapital_LondonStockEx_BUR
Quantity: 853486.0
CostPrice: 14.06

Instrument Name: EKFDiagnostics_LondonStockEx_EKF
Quantity: 925925.0
CostPrice: 0.27

Instrument Name: GBP_Cash
Quantity: 150000.0
CostPrice: 1.0




Portfolio: client-B-strategy-b

Now that we have imported our report, we can create holding adjustments like we did when we first populated our portfolios. These will post adjustment transactions to bring our portfolios in line with the fund accountant's records.

*Run the cell below to make the required adjustments to bring our fund-accountant-records scope in line with the fund accountant's daily report*

In [13]:
today = datetime.now(pytz.UTC)
t = time(hour=6, minute=30)
this_morning = pytz.utc.localize(datetime.combine(today, t)).isoformat()
    
for portfolio_name, portfolio in fund_accountant_daily_holdings_report.items():

    # Create a key and initialise a list to hold our adjustments for each portfolio
    holding_adjustments = []

    # Iterate over the holdings in each portfolio
    for instrument_name, holding in portfolio.items():
        # Get our Lusid Instrument Id
        if 'Cash' in instrument_name:
            identifier_key = 'Instrument/default/Currency'
        else:
            identifier_key = 'Instrument/default/LusidInstrumentId'
            
        Luid = instrument_universe[instrument_name]['identifiers']['LUID']
        
        # Create our adjust holdings request using our instrument universe to get the LUID identifier for the instrument
        holding_adjustments.append(
            models.AdjustHoldingRequest(
                instrument_identifiers={
                    identifier_key: Luid},
                tax_lots=[
                    models.TargetTaxLotRequest(units=holding['quantity'],
                                               cost=models.CurrencyAndAmount(
                                                   amount=holding['quantity'] *
                                                          holding['price'],
                                                   currency=instrument_universe[instrument_name][
                                                       'currency']),
                                               portfolio_cost=holding['quantity'] *
                                                              holding['price'],
                                               price=holding['price'])
                ])
        )
        
    response = client.transaction_portfolios.adjust_holdings(
        scope=fund_accountant_scope_code,
        code=portfolio_name,
        effective_at=this_morning,
        holding_adjustments=holding_adjustments)
        
    prettyprint.adjust_holdings_response(
        response, 
        fund_accountant_scope_code, 
        portfolio_name)
        
print ('All holdings adjusted')

Holdings Successfully Adjusted for Portfolio
Scope: fund-accountant-records-378c-6794-3521-3a
Code: client-A-strategy-balanced
Adjusted Holdings Effective From: 2019-07-19 06:30:00+00:00
Adjusted Holdings Created On: 2019-07-19 15:27:20.269690+00:00

Holdings Successfully Adjusted for Portfolio
Scope: fund-accountant-records-378c-6794-3521-3a
Code: client-A-strategy-tech
Adjusted Holdings Effective From: 2019-07-19 06:30:00+00:00
Adjusted Holdings Created On: 2019-07-19 15:27:20.741929+00:00

Holdings Successfully Adjusted for Portfolio
Scope: fund-accountant-records-378c-6794-3521-3a
Code: client-A-strategy-growth
Adjusted Holdings Effective From: 2019-07-19 06:30:00+00:00
Adjusted Holdings Created On: 2019-07-19 15:27:21.192615+00:00

Holdings Successfully Adjusted for Portfolio
Scope: fund-accountant-records-378c-6794-3521-3a
Code: client-B-strategy-balanced
Adjusted Holdings Effective From: 2019-07-19 06:30:00+00:00
Adjusted Holdings Created On: 2019-07-19 15:27:21.669528+00:00

Ho

Off the back of this adjustment we now have an updated set of portfolios in the fund accountant scope. We are unsure if these reconcile with our internal records or not.

![Scopes](img/transparency-fundaccountantreport.gif)

# 5) Reconcile our internal records with those of the fund accountant

Now that we have the fund accountant scope updated with this morning's report, we need to see how different the fund accountant's view of our position is with our own internal records.

We can do this by reconciling across all the portfolios inside the two scopes. We can only reconcile one portfolio at a time.

We can complete a reconciliation by defining the two portfolios to compare as well as the effective date and the as at date to use to view the holdings of each portfolio.

LUSID uses a bi-temporal data store. This means that not only can you reconcile two portfolios as they look today, you can also compare how one looks today, compared with how the other looked last week.

This makes reconciliation much easier and we will take advantage of this if we come across any discrepancies between our internal accounts and fund accountant's records.

*Run the cell below to complete reconciliations across all of our portfolios*

In [14]:
# Initialise our reconiled portfolios to dictionary to hold any reconciliation breaks
reconciled_portfolios = {}

# Iterate over our portfolios
for portfolio_name in fund_accountant_daily_holdings_report:

    # Define our internal portfolio
    internal_portfolio = models.PortfolioReconciliationRequest(
        portfolio_id=models.ResourceId(
            scope=internal_scope_code,
            code=portfolio_name),
        effective_at=this_morning,
        as_at=datetime.now(pytz.UTC).isoformat())

    # Define our fund accountant portfolio
    fund_accountant_portfolio = models.PortfolioReconciliationRequest(
        portfolio_id=models.ResourceId(
            scope=fund_accountant_scope_code,
            code=portfolio_name),
        effective_at=this_morning,
        as_at=datetime.now(pytz.UTC).isoformat())

    # Create our reconciliation request
    reconcile_holdings_request = models.PortfoliosReconciliationRequest(
        left=internal_portfolio,
        right=fund_accountant_portfolio,
        instrument_property_keys=["Instrument/default/Name"])

    # Reconcile the two portfolios
    reconciliation = client.reconciliations.reconcile_holdings(
        request=reconcile_holdings_request)

    # If there are any breaks, add them all to our dictionary
    if len(reconciliation.values) > 0:
        reconciled_portfolios[portfolio_name] = reconciliation
            
prettyprint.reconciliation(reconciled_portfolios, )

Portfolio: client-B-strategy-balanced

Reconciliation Break
Instrument: LUID_6P89F05F
Internal Units: 87982.0
Fund Accountant Units: 77481.0
Difference In Units: -10501.0
Internal Cost: 12367805.7
Fund Accountant Cost: 10891659.13
Difference In Cost: -1476146.57
Currency: GBP
Instrument/default/Name: UKGiltTreasury_4.5_2034


Reconciliation Break
Instrument: CCY_GBP
Internal Units: 237978.4280000001
Fund Accountant Units: 1714125.0
Difference In Units: 1476146.572
Internal Cost: 237978.4280000001
Fund Accountant Cost: 1714125.0
Difference In Cost: 1476146.572
Currency: GBP
Instrument/default/Name: CCY_GBP


Portfolio: client-C-strategy-balanced

Reconciliation Break
Instrument: LUID_ANQFWLK4
Internal Units: 1864441.0
Fund Accountant Units: 2206441.0
Difference In Units: 342000.0
Internal Cost: 3675745.43
Fund Accountant Cost: 4349998.43
Difference In Cost: 674253.0
Currency: GBP
Instrument/default/Name: TESCO PLC


Reconciliation Break
Instrument: CCY_GBP
Internal Units: 17937283.0
Fun

Okay so looking over our reconciliations we can see that 2 of our portfolios do not reconcile. This is Client A's balanced portfolio and Client B's balanced portfolio. All other portfolios are in sync and so we have coloured them light blue again.

![Scopes](img/transparency-reconciliationbreak.gif)

Looking over the reconciliation breaks above we can see that the discrepancy in Client A's balanced portfolio is due to our fund accountant portfolio having 10501 fewer units of an instrument than our own records. We have 87982 units of this instrument according to our records, however the fund accountant only has 77481 units. There is also a difference in the cash balance in the other direction.

This seems to indicate that perhaps a Buy transaction has not gone through.

For Client B's balanced portfolio we can see that the discrepancy is due to our fund accountant portfolio having 342000 more units of an instrument than our own records. We have 1,864,441 units of this instrument according to our records, and the fund accountant has 2,206,441 units. Once again there is also a difference in the cash balance in the opposite direction.

This seems to indicatethat perhaps a Sell transaction has not gone through.

So perhaps in the fund accountant's record there is a Buy transaction missing from Client A's balanced portfolio and a Sell transaction missing from Client B's balanced portfolio. We would now need to try and identify the transaction, perhaps using a spreadsheet or similar tool.

This is the sort of manual work that our 'shadow' accountant team and investment managers are currently having to do manually every morning. Let's see if we can automate this.

# 6) Build a system to automatically identify and flag discrepancies

So we have identified a number of discrepancies between our internal records and the fund accountant's records. It seems as though perhaps a buy and a sell transaction have not gone through.

We do know that the fund accountant finalise their accounts at the close of trade which is 8 hours after trading opens each day. Any transactions posted after that won't appear in the next morning's report. Perhaps we had some late trades yesterday which caused the issue.

We can wind back the clock to the close of trade yesterday and see how our two records compare.

*Run the cell below to wind back the clock to the close of trade yesterday and compare our two books of record*

In [15]:
trade_close_time = (yesterday_trade_open + timedelta(hours=8)).isoformat()

# Initialise our dictionary to hold our reconciliation breaks
reconciled_portfolios_trade_close = {}
    
# Iterate over our portfolios
for portfolio_name in fund_accountant_daily_holdings_report:

    # Internal portfolio
    internal_portfolio = models.PortfolioReconciliationRequest(
        portfolio_id=models.ResourceId(
            scope=internal_scope_code,
            code=portfolio_name),
        effective_at=trade_close_time,
        as_at=datetime.now(pytz.UTC).isoformat())
    
    # Fund accountant portfolio
    fund_accountant_portfolio = models.PortfolioReconciliationRequest(
        portfolio_id=models.ResourceId(
            scope=fund_accountant_scope_code,
            code=portfolio_name),
        effective_at=this_morning,
        as_at=datetime.now(pytz.UTC).isoformat())

    # Create our reconciliation request
    reconcile_holdings_request = models.PortfoliosReconciliationRequest(
        left=internal_portfolio,
        right=fund_accountant_portfolio,
        instrument_property_keys=[])

    # Reconcile our two portfolios
    reconciliation = client.reconciliations.reconcile_holdings(
        request=reconcile_holdings_request)

    # If there are any breaks, add them all to our dictionary
    if len(reconciliation.values) > 0:
        reconciled_portfolios_trade_close[portfolio_name] = reconciliation

prettyprint.reconciliation(reconciled_portfolios_trade_close, True)

No reconciliation breaks


Here we can see that the two portfolios reconcile perfectly. Let's find out what happened after trading. So any divergence between our records and those of the fund accountant happened after the close of trading. Let's take a look at what transactions happened after this time.

We can do this by looping over our portfolios and retrieving any transactions that happened between the close of trade and this morning before trade begins.

*Run the cell below to identify late trades*

In [16]:
late_trades = {}

for portfolio_name in fund_accountant_daily_holdings_report:

    late_trade = client.transaction_portfolios.get_transactions(
        scope=internal_scope_code,
        code=portfolio_name,
        from_transaction_date=trade_close_time,
        to_transaction_date=this_morning)
    
    if len(late_trade.values) > 0:
        late_trades[portfolio_name] = late_trade

prettyprint.trades(late_trades)

Portfolio: client-B-strategy-balanced

Transaction: tid_6fc1a588-f19b-4222-93d5-c35dcf4c7d46
Instrument: LUID_6P89F05F
Type: Buy
Units: 10501.0
Price: 140.572
Currency: GBP
Transaction Date: 2019-07-18 16:12:00+00:00


Portfolio: client-C-strategy-balanced

Transaction: tid_adefc478-aca8-4a9b-89b1-57f49a31d1b5
Instrument: LUID_ANQFWLK4
Type: Sell
Units: 342000.0
Price: 1.8865
Currency: GBP
Transaction Date: 2019-07-18 16:18:00+00:00




We have identifed transactions made after the close of trade

![Scopes](img/transparency-latetrades.gif)

We can see that there were two lates trades yesterday. Let's see if these transactions match up with our reconciliation breaks. We can do this by checking if the instrument Id and the number of units match. If so we can be reasonably confident that the late trade is the cause of the reconciliation break. We can add further matching details to increase our confidence as we see fit.

*Run the cell below to see if our late trades match with either of our reconciliation breaks*

In [17]:
# Initialise our dictionary to hold our exceptions
matched_exceptions = []
  
# Iterate over our portfolios and the late trades we have identified
for portfolio_name, late_trade in late_trades.items():
    # Loop over each late trade
    for trade in late_trade.values:
        # Check if it matches a reconciliation break
        for reconciliation_break in reconciled_portfolios[portfolio_name].values:
            '''
            Here we use the absolute difference in units to reduce the complexity in identifying if a
            transaction is a buy or a sell and whether we need to match a positive or negative difference in
            units. In practice we can add in the ability to check the direction as well
            '''
            units = abs(reconciliation_break.difference_units)
            instrument_uid = reconciliation_break.instrument_uid

            # If the instrument id and units match, we have identified the cause of the reconciliation break
            if (trade.instrument_uid == instrument_uid) and (trade.units == units):
                matched_exceptions.append((trade, reconciliation_break, portfolio_name))

prettyprint.exceptions(matched_exceptions)

Transaction: tid_6fc1a588-f19b-4222-93d5-c35dcf4c7d46
Instrument: LUID_6P89F05F
Type: Buy
Units: 10501.0
Price: 140.572
Currency: GBP

MATCHES

Reconciliation Break
Instrument: LUID_6P89F05F
Internal Units: 87982.0
Fund Accountant Units: 77481.0
Difference In Units: -10501.0
Internal Cost: 12367805.7
Fund Accountant Cost: 10891659.13
Difference In Cost: -1476146.57
Currency: GBP



Transaction: tid_adefc478-aca8-4a9b-89b1-57f49a31d1b5
Instrument: LUID_ANQFWLK4
Type: Sell
Units: 342000.0
Price: 1.8865
Currency: GBP

MATCHES

Reconciliation Break
Instrument: LUID_ANQFWLK4
Internal Units: 1864441.0
Fund Accountant Units: 2206441.0
Difference In Units: 342000.0
Internal Cost: 3675745.43
Fund Accountant Cost: 4349998.43
Difference In Cost: 674253.0
Currency: GBP





Here we can see that our two late trades match the instrument reconciliation breaks that we have identified.

Now that we have identified the reason for the reconciliation breaks, we want to flag the transactions in question so that we can confirm using tomorrow's report whether or not they have finally been included in the fund accountant's reporting.

To do this we can use LUSID properties. We can create properties for any object inside LUSID. These properties are defined by us in advance.

In this case we will create a property called 'late_trade' which will be a boolean property that is True if the trade has been flagged as a late trade. We will make it so that if we add this property to a trade it is required to have the value.

When we set up a property we are required to give it a data type. We do this by specifying a pre-configured data type using its ResourceId. Once again this is the scope and code of the object. There are a number of pre-configured types in the default scope which we can draw from. In this case we will use the boolean type. This means that when we set the 'late_trade' property we have to specify a value of 'True' or 'False'.

*Run the cell below to set a late_trade = True property on our late trades*

In [18]:
property = models.CreatePropertyDefinitionRequest(
    domain='Transaction',
    scope=internal_scope_code,
    code='late_trade',
    value_required=True,
    display_name='late_trade',
    data_type_id=models.ResourceId(
        scope='system',
        code='boolean'))

client.property_definitions.create_property_definition(
    definition=property)

for exception in matched_exceptions:
    
    transaction_id = exception[0].transaction_id
    portfolio_name = exception[2]
    
    r = client.transaction_portfolios.add_transaction_property(
        scope=internal_scope_code,
        code=portfolio_name,
        transaction_id=transaction_id,
        transaction_properties={
            'Transaction/{}/late_trade'.format(internal_scope_code): 
                                    models.PropertyValue(label_value='True')
        })

    prettyprint.add_property_response(
        r, 
        internal_scope_code, 
        portfolio_name, 
        transaction_id)

Property Successfully Added On Transaction in Portfolio
Transaction: tid_6fc1a588-f19b-4222-93d5-c35dcf4c7d46
Scope: internal-records-378c-6794-3518-5a
Code: client-B-strategy-balanced
Property Effective From: 2019-07-18 16:12:00+00:00
Property Created On: 2019-07-19 15:27:34.673459+00:00

Property Successfully Added On Transaction in Portfolio
Transaction: tid_adefc478-aca8-4a9b-89b1-57f49a31d1b5
Scope: internal-records-378c-6794-3518-5a
Code: client-C-strategy-balanced
Property Effective From: 2019-07-18 16:18:00+00:00
Property Created On: 2019-07-19 15:27:34.958974+00:00



We now have the ability to automatically update our fund accountant record each day with their daily report, reconcile their understanding of our holdings against our own and automatically identify any discrepancies caused by late trades.

We can build on this to automatically identify and flag other causes of discrepencies. This will allow us to reduce the resources assigned to our 'shadow' accounting team and free up our portfolio managers.

Our final state is shown in the diagram below.

![Scopes](img/transparency-matchedexceptions.gif)